In [1]:
%reload_ext autoreload
%autoreload 2

from interactive import utils, config, visualisation

Config = config.Config()

Updating tessera-manifests repository in C:\Users\Artyom Gabtraupov\AppData\Local\geotessera\geotessera\Cache\tessera-manifests
✓ tessera-manifests updated to latest version (c824fd053)


# Welcome to TESSERA!

In this interactive notebook you can visualise the embeddings for a region of interest (ROI) and train a [k-Nearest Neigbours](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) classifier.

We hope this will demonstrate the power of TESSERA's openly-available embeddings for performing land cover classification over a range of areas and cover classes.

### How to use

1. Run the previous cell (by clicking the 'play' button) to load the necessary code
2. Update your region of interest (ROI) in the cell below
3. Run the cell below!
4. When the map appears:
   - Navigate to your region of interest (zoom/pan)
   - Click the rectangle tool in the drawing toolbar
   - Draw a rectangle on the map
   - Click 'Get Coordinates'
5. Run the next cell to visualise and classify the TESSERA embeddings:
   - Follow the interface to select points on the map for each class
   - Add new classes as necessary
   - Click 'classify' to see the result of the k-Nearest Neigbours classifier
   - Export your points by clicking 'Save labels'

This product is in early stages: please get in touch with any feedback.

In [2]:
# -- 1. ROI DEFINITION --

# Create and display the bounding box selector
bbox_selector = visualisation.BoundingBoxSelector()
bbox_selector.display()

In [3]:
bbox_tuple = bbox_selector.get_bbox()
if bbox_tuple:
    (MIN_LAT, MAX_LAT), (MIN_LON, MAX_LON) = bbox_tuple
else:
    raise ValueError("No bounding box selected")

# -- 2. FETCH AND MOSAIC RELEVANT TESSERA TILES --

embedding_mosaic, mosaic_transform = utils.TesseraUtils().process_roi_to_mosaic(
    lat_coords=(MIN_LAT, MAX_LAT),
    lon_coords=(MIN_LON, MAX_LON),
)

# -- 3. VISUALISE MAP, PLACE TRAINING POINTS, & RUN CLASSIFICATION --

mapping_tool = visualisation.InteractiveMappingTool(
    MIN_LAT,
    MAX_LAT,
    MIN_LON,
    MAX_LON,
    embedding_mosaic,
    mosaic_transform,
)
mapping_tool.display()

Updating tessera-manifests repository in C:\Users\Artyom Gabtraupov\AppData\Local\geotessera\geotessera\Cache\tessera-manifests
✓ tessera-manifests updated to latest version (c824fd053)
Bounding box defined:
┗ (51.50°, -0.14°) ┓ (51.51°, -0.11°)

Fetching embeddings for ROI: (-0.137329, 51.501316, -0.113382, 51.510078) for year 2024
Loading 1 registry blocks for region bounds: (-0.1373, 51.5013, -0.1134, 51.5101)
Successfully loaded 1 new registry blocks (1/1 total available)
Tile 1/1: Downloading 0.15°W, 51.55°N (100%) (100/100)
Tile 1/1: Downloaded 0.15°W, 51.55°N (100/100)
Tile 1/1: Downloading 0.15°W, 51.55°N (100%) (100/100)
Tile 1/1: Downloaded 0.15°W, 51.55°N (100/100)
Completed tile 1/1 (100/100)
Fetched 1 tiles. Creating mosaic...


Processing tiles:   0%|          | 0/1 [00:00<?, ?it/s]


Merging all tiles...
Shape of final embedding mosaic: (983, 1055, 128)
Bounds of displayed embedding mosaic: ┗ (51.50, -0.21) | ┓ (51.60, -0.09)

Creating PCA-based visualization...
Normalizing PCA components for display...
PCA visualization created.
Image overlay added to map.
